<a href="https://colab.research.google.com/github/earendil94/SMLExam/blob/preferivolescimmie/StillExploringSMLExam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## A quick overview

**Q) What still needs to be done regarding the RNN?**

A) We still need to find a proper way to sample an image and a caption.
I wanted to try to get the RNN to work alone but I am missing the proper
label if I use the text as an input. Moreover we need to define the learning procedure of our model

#### Useful stuff
https://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/

https://towardsdatascience.com/automatic-image-captioning-with-cnn-rnn-aae3cd442d83




In [6]:
#Installing things in colab
!pip install spacy
!python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.6/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.6/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [13]:
#Import

import torch
from torch import nn
from torch import optim
import pandas as pd
from spacy.lang.en import English #I am not even sure we need it. In fact I believe we don't
import en_core_web_sm

In [14]:
#Some utils: we'd better turn them into a package later on
#Still need to understand whether we need more of this or not

class Vocab:
  
  '''
  
  '''

  def __init__(self, corpus):
    self.start_word = "<Start>"
    self.end_word = "<End>"
    self.corpus = corpus
  
  def __

class DecoderRNN(nn.Module): 
  def __init__(self, embed_size, hidden_size, vocab_size, num_layers = 1): #Here we define the layers
    super().__init__()
    self.embedding_captions_layer = nn.Embedding(vocab_size, embed_size)
    self.LSTM = nn.LSTM(input_size = embed_size, hidden_size = hidden_size, 
                        num_layers = num_layers, batch_first = TRUE)
    self.linear = nn.Linear(hidden_size, vocab_size)

  def forward(self, features, captions): #Notice that features will only be used when we will have the encoder images in input
    #captions = captions[:, :-1]
    embed = self.embedding_layer(captions)
    #A couple of comments here: torch.cat concatenates just like in unix
    #torch.unsqueeze instead transforms the tensor into a column vector (column since we specify 1 here)
    #embed = torch.cat((features.unsqueeze(1), embed), dim = 1) 
    lstm_outputs, _ = self.lstm(embed)
    out = self.linear(lstm_outputs)




In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [17]:
#Loading captions file and creating the array

def load_vocab(path):
  df = pd.read_csv(path, sep="|")
  captions_array = df[' comment']
  captions_array.fillna("", inplace=True)
  captions_array = df[' comment'].values
  captions_array = captions_array[:200]
  captions = []

  for i in captions_array:
    captions.append(i)

  nlp = English()
  tokens = []
  voc = Vocab()

  tokens.append(voc.start_word)
  tokens.append(voc.end_word)

  for sentence in captions:
    text = nlp(sentence)
    for tk in text:
      tokens.append(tk.text)

  word_list = sorted(set(tokens))
  vocab_size = len(word_list)
  return vocab_size


In [4]:
#Actual run
embed_size = 100
hidden_size = 100
epochs = 3
path = "/content/drive/My Drive/SML_Project/Chunk1/results.csv"
learning_rate = 0.01


vocab_size = load_vocab(path=path)
decoder = DecoderRNN(embed_size=embed_size, hidden_size=hidden_size, vocab_size=vocab_size)
device = "cpu"

decoder.to(device)
criterion = nn.CrossEntropyLoss()
params = decoder.parameters()
optimizer = optim.Adam(lr = learning_rate)


Embedding(2, 3)